In [1]:
# In linear networks, PDLT says that covariance for activation at some neuron
# will be $G^{(l)}_2 = C_W^l 1/n_0 \sum_{j=1}^{n_0} x_{j}^2
#
# The intralayer interaction of the magnitudes then is
# E[(z_j z_j - G_2)(z_k z_k - G_2)] = G_4 - G_2^2 = 2(l-1)/n G_2^2

# TODO: Vary l and n to see around when the binomial approximation really breaks down

In [63]:
import itertools
from collections import defaultdict

import numpy as np
import torch
from tqdm import tqdm

In [123]:
# So we'll randomly initialize an input vector
# Then randomly initialize the network 1_000 times and store the activation at each
# layer. Then we'll compute the covariances of these activations

input_dim = 10
# 2 inputs
n_inputs = 2
input = torch.randn((n_inputs, input_dim))
input /= input.norm(dim=1, keepdim=True)

attempts = 1000
n = 1000
L = 3
C_W = 100

In [139]:
input[0].norm(dim=0, keepdim=True)

tensor([1.])

In [124]:
def initialize_model(n, L, C_W):
    layers = []
    for i in range(l):
        first_dim = input_dim if i == 0 else n
        layer = torch.nn.Linear(first_dim, n)
        # initialize layer
        torch.nn.init.normal_(layer.weight, std=(C_W / first_dim) ** 0.5)
        torch.nn.init.zeros_(layer.bias)
        layers.append(layer)

    model = torch.nn.Sequential(*layers)
    return model

In [125]:
activations_per_layer = defaultdict(list)
for attempt in tqdm(range(attempts)):
    x = input
    model = initialize_model(n, L, C_W)

    # Forward pass and compute covariance (passing layer by layer to avoid using hooks)
    for i, layer in enumerate(model):
        x = layer(x)
        activations_per_layer[i].append(x.detach())

for i, _ in enumerate(model):
    activations_per_layer[i] = torch.stack(activations_per_layer[i])
    assert activations_per_layer[i].shape == (attempts, n_inputs, n)

100%|██████████| 1000/1000 [00:18<00:00, 55.45it/s]


In [126]:
len(activations_per_layer), len(activations_per_layer[0])

(3, 1000)

In [127]:
def covariance(sample_1, sample_2, activations):
    # Covariance of activations in some layer
    act_a, act_b = (
        activations[:, sample_1, :],
        activations[:, sample_2, :],
    )  # 2 (attempts, 1, neurons)
    act_a, act_b = act_a.squeeze(1), act_b.squeeze(1)  # 2 (attempts, neurons)
    # Now do E[(X-E[X]) (Y-E[Y])]
    act_a = act_a - act_a.mean(dim=0, keepdim=True)
    act_b = act_b - act_b.mean(dim=0, keepdim=True)
    cov = (act_a * act_b).mean(dim=0)
    return cov

In [128]:
covariances = {}

for sample_1, sample_2 in itertools.product((0, 1), repeat=2):
    covariances[(sample_1, sample_2)] = {}
    for layer, activations in activations_per_layer.items():
        cov = covariance(sample_1, sample_2, activations)
        covariances[(sample_1, sample_2)][layer] = cov

In [137]:
def inner_product(sample_1, sample_2, input):
    # return ((input[sample_1] @ input[sample_2]) / input.shape[1]).item()
    return (
        sum(i1 * i2 for i1, i2 in zip(input[sample_1], input[sample_2])) / input.shape[1]
    ).item()


inner_product(0, 0, input)

0.10000000149011612

In [131]:
# If C_W was small, we expect covariance in output to be tiny always
samples = (1, 0)
max_ = max(abs(covariances[samples][L - 1])).item()
ip = inner_product(*samples, input)
C_W, max_, ip, max_ / ip

(100, 114729.1015625, 0.10000000149011612, 1147290.9985290319)

In [13]:
# Off-diagonal ones should be 0
for layer, cov in enumerate(covariances):
    maximum_deviation = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                var = cov[i][j]
                if var > maximum_deviation:
                    maximum_deviation = var
    print(f"{layer=} {maximum_deviation=}")

layer=0 maximum_deviation=0.0013306538234676723
layer=1 maximum_deviation=1.4527258799732309e-05
layer=2 maximum_deviation=1.5048692425172388e-07


In [88]:
input[0]

tensor([ 0.8131,  0.6417,  0.1147, -1.4479, -0.6727,  0.7635,  1.6740,  0.0495,
        -0.0669,  0.2204])

In [91]:
# What's inner product of input?
inner_product = sum(component**2 for component in input[0].numpy().flatten()) / len(
    input[0].numpy().flatten()
)
inner_product

0.7075738648175647

In [15]:
# Diagonal entries should scale with C_W
for layer, cov in enumerate(covariances):
    maximum_deviation = 0
    for i in range(n):
        abs_deviation = abs(cov[i][i] - C_W**layer * inner_product)
        if abs_deviation > maximum_deviation:
            maximum_deviation = abs_deviation
    print(f"{layer=} {maximum_deviation=}")

layer=0 maximum_deviation=0.8907861530065746
layer=1 maximum_deviation=0.008907744538290272
layer=2 maximum_deviation=8.906750944679384e-05


In [ ]:
# What's

In [122]:
# Now let's try the fourth cumulant
# For every pair of activations per layer, subtract the covariance of each activation
# and get product of deviations
fourth_cumulant = []
for layer in range(l):
    print(f"{layer=}")
    cov = covariances[layer]
    fourth_cumulant.append([])
    # Let's just use the first attempt
    activations = activations_per_layer[layer][0]
    # Choose two disjoint neurons
    maximum_ = 0
    i_, j_ = (0, 0)
    for i in range(n):
        fourth_cumulant[layer].append([])
        for j in range(n):
            deviation_i = activations[i] ** 2 - cov[i][i]
            deviation_j = activations[j] ** 2 - cov[j][j]
            prod = deviation_i * deviation_j
            if prod > maximum_ and i != j:
                maximum_ = prod
                i_, j_ = i, j

            fourth_cumulant[layer][i].append(prod)
    print(maximum_, i_, j_)

    fourth_cumulant[layer] = np.array(fourth_cumulant[layer])

layer=0
38.866870436205375 115 538
layer=1
21.8250891684642 125 766
layer=2
12.861546160345714 4 842


In [123]:
fourth_cumulant[0].shape

(1000, 1000)

In [125]:
# Off-diagonal ones should be 2(l-1)/n * G_2^2
for layer in range(l):
    print(f"{layer=}")
    cov = covariances[layer]
    deviations_products = fourth_cumulant[layer]

    expected_correlation = 2 * (layer - 1) / n * (C_W**layer * inner_product) ** 2

    total_deviations_sum = 0
    for i in range(n):
        for j in range(n):
            if i != j:
                total_deviations_sum += deviations_products[i][j]
    mean_deviation = total_deviations_sum / (n**2 - n)
    print(f"{layer=} {mean_deviation=} {expected_correlation}")

layer=0
layer=0 mean_deviation=-0.0003021667286407805 -0.00031977996299061256
layer=1
layer=1 mean_deviation=7.721738728385593e-05 0.0
layer=2
layer=2 mean_deviation=0.00023255435845350234 0.00031977996299061256
